In [ ]:
import pydicom
import numpy as np
import matplotlib.pyplot as plt

# Path to your DICOM file
dcm_file_path = "data/radioprotect/Rakathon Data/SAMPLE_004/RS.1.2.246.352.221.469828402979830434171132323292476061.dcm"

# Load the DICOM file
try:
    dcm = pydicom.dcmread(dcm_file_path)
except Exception as e:
    print(f"Error loading DICOM file: {e}")
    exit()

In [ ]:
# Extract pixel data as a NumPy array
if hasattr(dcm, "pixel_array"):
    image = dcm.pixel_array
    print(f"Image shape: {image.shape}")pydicom
    print("No pixel data found in the DICOM file.")
    exit()

# Optional: Display the CT image
plt.imshow(image, cmap="gray")
plt.title("CT Scan")
plt.axis("off")  # Hide axes
plt.show()

# Optional: Access metadata (e.g., Patient ID, Slice Thickness)
print(f"Patient ID: {dcm.get('PatientID', 'N/A')}")
print(f"Slice Thickness: {dcm.get('SliceThickness', 'N/A')} mm")


In [ ]:
dcm.keys

In [ ]:
from pathlib import Path

p = Path('/Volumes/Transfer/radioprotect/Rackaton Data/SAMPLE_002')

contents = list(p.iterdir())

In [ ]:
for c in contents:
    if "dcm" not in c.name:
        print(c.name)

In [ ]:
import os
import pydicom
import sys


def find_rtstruct_files(folder_path):
    """
    Traverses a folder and identifies DICOM files with Modality 'RTSTRUCT'.

    Args:
        folder_path (str): The path to the folder containing DICOM files.

    Returns:
        list: A list of file paths identified as RTSTRUCT files.
    """
    rtstruct_files = []
    print(f"Searching for RT Structure Set (RTSTRUCT) files in: {folder_path}\n" + "=" * 30)

    # Walk through the directory tree
    for root, dirs, files in os.walk(folder_path):
        for filename in files:
            # Check if the file might be a DICOM file (basic check)
            if filename.lower().endswith(".dcm"):
                filepath = os.path.join(root, filename)
                try:
                    # Read the DICOM file metadata only (faster)
                    # We don't need pixel data to check the modality
                    ds = pydicom.dcmread(filepath, stop_before_pixels=True)

                    # Check if the Modality tag exists and is 'RTSTRUCT'
                    # Using getattr is safer than direct access in case the tag is missing
                    if getattr(ds, "Modality", None) == "RTSTRUCT":
                        print(f"[*] Found RTSTRUCT file: {filepath}")
                        rtstruct_files.append(filepath)

                except pydicom.errors.InvalidDicomError:
                    # Silently ignore files that are not valid DICOM format
                    # print(f"[!] Skipping non-DICOM or invalid file: {filepath}")
                    pass
                except Exception as e:
                    # Catch other potential errors during file reading
                    print(f"[!] Error reading {filepath}: {e}")

    print("=" * 30 + f"\nSearch complete. Found {len(rtstruct_files)} RTSTRUCT file(s).")
    return rtstruct_files


# --- How to run the script ---
if __name__ == "__main__":
    # *** IMPORTANT: Set the path to your DICOM folder here ***
    # Example: target_directory = "/mnt/data/dicom_dump"
    # Example: target_directory = "C:\\Users\\YourUser\\Desktop\\DICOM_Data"
    target_directory = "./data/radioprotect/Rackaton Data/SAMPLE_001"  # Default: Searches the current directory where the script is 
    if not os.path.isdir(target_directory):
        print(f"\nError: The specified folder does not exist: {target_directory}")
    else:
        found_files = find_rtstruct_files(target_directory)
        # You can now use the 'found_files' list if needed
        # For example:
        # if found_files:
        #     first_rtstruct = found_files[0]
        #     print(f"\nProcessing the first found file: {first_rtstruct}")
        #     # Add code here to process the RTSTRUCT file
